In [13]:
!pip install datasets transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


  Obtaining dependency information for kmedoids from https://files.pythonhosted.org/packages/6e/b4/457e552aa362cc746fbf0c461ab1a9bd7794db3321f82586601769f27112/kmedoids-0.5.3.1-cp312-cp312-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 kB 9.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Setup

In [1]:
%load_ext autoreload
%autoreload 2

device='mps'
from transformers import ClapModel, ClapFeatureExtractor
model = ClapModel.from_pretrained("laion/clap-htsat-unfused", use_safetensors=True).to(device)
feature_extractor: ClapFeatureExtractor = ClapFeatureExtractor.from_pretrained("laion/clap-htsat-unfused")


# Run

In [2]:
import torchaudio
import torchaudio.transforms as T

filename = '/Users/damian/Downloads/Dua Lipa - Be The One (Official Music Video).mp3'
waveform, sampling_rate = torchaudio.load(filename)

if sampling_rate != feature_extractor.sampling_rate:
    resampler = T.Resample(sampling_rate,  feature_extractor.sampling_rate, dtype=waveform.dtype)
    waveform, sampling_rate = resampler(waveform), feature_extractor.sampling_rate

print(sampling_rate, waveform.shape)
#inputs = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt")
#audio_features = model.get_audio_features(**inputs)

48000 torch.Size([2, 9805880])


In [31]:
from typing import Generator
import torch
torch.set_grad_enabled(False)
from tqdm.auto import tqdm

def get_audio_chunks(waveform, chunk_size_seconds: float=0.25) -> Generator[torch.Tensor, None, None]:
    if len(waveform.shape) != 1:
        raise ValueError("waveform should have shape [num_samples]")
    chunk_size = int(8 * ((sampling_rate * chunk_size_seconds) // 8))
    for offset in range(0, waveform.shape[0], chunk_size):
        yield waveform[offset:offset+chunk_size]
    

def get_features_chunked(waveform: torch.Tensor, sampling_rate: int, chunk_size_seconds: float=0.25) -> torch.Tensor:
    all_features = []
    for chunk in get_audio_chunks(waveform, chunk_size_seconds=chunk_size_seconds):
        inputs = feature_extractor(chunk, sampling_rate=sampling_rate, return_tensors="pt")
        #print(inputs.keys())
        audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
        all_features.append(audio_features)
        
    return torch.concat(all_features)

def get_features(waveform: torch.Tensor, sampling_rate):
    inputs = feature_extractor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    #print(inputs.keys())
    audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
    return audio_features



In [64]:
bpm = 87.5/4
chunk_size_seconds = 60/bpm
left_chunks = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds))[:-1]
right_chunks = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds))[:-1]
mono_chunks = [(left_chunks[i] + right_chunks[i]) / 2 for i in range(len(left_chunks))]
all_features = torch.concat([get_features(chunk, sampling_rate=sampling_rate) 
                             for chunk in tqdm(mono_chunks)])



  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

In [65]:

import pickle
with open(filename + f'.clap-cs{chunk_size_seconds}.pkl', 'wb') as f:
    pickle.dump(all_features, f)


In [66]:
from src.clap_slice.medoids_tsp import sort_tsp
order = sort_tsp(all_features)

computing distance matrix
computing route


In [67]:
chunks_sorted = [torch.stack([left_chunks[index], right_chunks[index]])
    for index in order]
print(chunks_sorted[0].shape)
sorted_audio = torch.cat(chunks_sorted, dim=1)
print(sorted_audio.shape)

torch.Size([2, 131656])
torch.Size([2, 9742544])


In [68]:
torchaudio.save(filename + f'-sorted-cs{chunk_size_seconds}.wav', sorted_audio, sample_rate=sampling_rate)

In [74]:
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    print(envelope)
    def wrap(idx):
        if idx < 0:
            return len(chunks) + idx
        elif idx >= len(chunks):
            return idx - len(chunks)
        else:
            return idx
    smeared_chunks = [torch.zeros_like(chunks[0]) 
                     for _ in range(len(chunks))]
    for source_chunk_idx, chunk in enumerate(chunks):
        for smear_slot in range(-smear_width, smear_width+1):
            smeared_chunks[wrap(source_chunk_idx+smear_slot)] += chunk * envelope[smear_slot+smear_width] * (1/smear_width)

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smeared_chunks = list(smear_2(chunks_sorted, 4))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-cs{chunk_size_seconds}-smeared.wav', smeared, sample_rate=sampling_rate)


tensor([0.0000, 0.1464, 0.5000, 0.8536, 1.0000, 0.8536, 0.5000, 0.1464, 0.0000])
torch.Size([2, 9742544])
